# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Name & GitHub

- Name: Alan Madrigal
- GitHub Username: alanmc979

# Research Question

Can it be predicted how likely a restaurant in San Diego is to violate the local food health policy, by looking at the relationship between food policy violations and the type of food each restaurant serves?

## Background and Prior Work


First Reference: I found a similar work to what I am looking for in the city of Chicago. They showed how they predict food inspection violations which are similar to what I want to accomplish for the city of San Diego. Their method found out that they were able to improve the inspection process by finding faster the places that had food policy violations. By finding violations faster they help reduce food poisoning cases in that area. 

Second Reference: In the second reference, I found how the city of San Diego makes inspects restaurants. 
- 1. Each violation of the Food Inspection Report is assigned a point value depending on its importance. For example, a Major Risk Factor is worth four points, a Minor Risk Factor is worth two, and a Good Retail Practice is worth one.
- 2. Once the Specialist completes an inspection, the points are added up and subtracted from 100. The resulting number is the inspection "score".
- 3. A letter grade is assigned to the facility based on the inspection score. An "A" grade means the facility earned a score of 90 to 100 percent and is in satisfactory compliance with state law; a "B" means the facility earned a score of 80 to 89 percent and needs improvement; a "C" means the facility earned a score of 79 percent or less and is a failing grade.
- 4. The grade card must be displayed near the public entrance during hours of operation. This information allows me to understand how data can be used. This information about the scoring of every place can be helpful for the prediction algorithm. 

Third Reference: 
    Food-borne illness affects an estimated 48 million Americans each year, resulting in 3,000 deaths and the hospitalization of 128,000 people, according to the Centers for Disease Control and Prevention (CDC). Beyond that, food-borne illness in the U.S. is enormously costly, with an estimated collective annual bill of $55 billion in medical treatment, lost productivity, and lost wages, not to mention litigation expenses. This article makes an emphasis on the impact of food poisoning and how important it is to prevent it. It also takes into consideration how inspection biases can hurt the inspection process. Hence we found that answering this data science question can help prevent food poisoning and improve the inspection process in San Diego. But we should focus on having accurate data from the inspectors without biases.

References (include links):
- 1) https://chicago.github.io/food-inspections-evaluation/
- 2)https://www.sandiegocounty.gov/content/sdc/deh/fhd/ffis/intro.html.html
- 3)https://hbr.org/2019/05/to-improve-food-inspections-change-the-way-theyre-scheduled

# Hypothesis


I believe that there is a relationship between the type of food a restaurant serves and their food violations. Food poisoning is more likely to happen with certain types of food, therefore I believe there must be a relation between food policy violations and the type of food the restaurant serves. Hence by detecting the restaurants that are more likely to violate food policies, we can prioritize the restaurants to inspect first and this should improve the inspection process.

# Dataset(s)


The ideal data set will have the following variables: Name (of the restaurant), Type of food (the restaurant serves), Food Inspection Grade (0-100) Number of previous year violations The data will be stored in multiple spreadsheets, each sheet for each year, which should have as many observations as the number of restaurants in San Diego. This can be stored in a CSV for easy manipulation with the Pandas module. By having all this information we can determine the relationship between the type of food and food policy violations, as well as to set a queue, with this information, for the order of the next restaurants’ inspection to find food policy violations faster.

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [52]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import collections
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns, numpy as np
import patsy

In [53]:
df_insp = pd.read_csv('/Users/alanmc/Desktop/data/inspections.csv')
#check out data 
df_insp.head()

,hsisid,date,name,address1,address2,city,state,postalcode,phonenumber,restaurantopendate,...,num_non_critical,num_critical_previous,num_non_critical_previous,num_critical_mean_previous,num_non_critical_mean_previous,avg_neighbor_num_critical,avg_neighbor_num_non_critical,top_match,second_match,critical
0,4092013748,2012-09-21T00:00:00Z,Cafe 3000 At Wake Med,3000 New Bern Ave,NaN,raleigh,NC,27610,(919) 350-8047,2002-12-21T00:00:00Z,...,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,4092014046,2012-09-21T00:00:00Z,Overtime Sports Pub,1030-149 N Rogers Ln,NaN,raleigh,NC,27610,(919) 255-9556,2004-05-04T00:00:00Z,...,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4092015191,2012-09-21T00:00:00Z,TASTE OF CHINA,6209 ROCK QUARRY RD,STE 126,raleigh,NC,27610,(919) 773-2285,2008-08-04T00:00:00Z,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,4092016122,2012-09-21T00:00:00Z,Panera Bread #1643,1065 Darrington DR,NaN,cary,NC,27513,NaN,2012-03-28T00:00:00Z,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,4092021513,2012-09-21T00:00:00Z,WalMart Supercenter #4499-00 Deli/Bakery,841 E Gannon AVE,NaN,zebulon,NC,27597,(919) 269-2221 ext. 304,2008-02-25T00:00:00Z,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [54]:
df_zip = pd.read_csv('/Users/alanmc/Desktop/data/zipcodes.csv')
df_zip.head()

,zip,median_family_income_dollars,median_household_income_dollars,per_capita_income_dollars,percent_damilies_below_poverty_line,percent_snap_benefits,percent_supplemental_security_income,percent_nonwhite
0,27501,59408,51121,21631,10.5,15.5,5.2,17.9
1,27502,109891,95857,36763,3.4,2.4,0.8,18.9
2,27511,82292,67392,33139,9.6,4.5,2.2,24.8
3,27513,109736,87262,41232,3.8,2.4,1.5,27.8
4,27518,125432,98247,49865,5.5,1.0,1.7,19.9


In [55]:
df_yelp = pd.read_csv('/Users/alanmc/Desktop/data/yelp.csv')
df_yelp.head()

,id,name,is_closed,rating,review_count,address1,zip_code,latitude,longitude,price,...,italian,newamerican,chicken_wings,delis,bars,salad,seafood,bbq,bakeries,sushi
0,42nd-street-oyster-bar-rdu-airport-morrisville,42nd street oyster bar - rdu airport,False,2.5,23,2400 John Brantley Blvd,27560,35.873808,-78.790896,$$,...,False,True,False,False,False,False,True,False,False,False
1,aldi-foods-cary,aldi foods,False,4.0,17,2303 NW Maynard Rd,27519,35.802840,-78.785930,$,...,False,False,False,False,False,False,False,False,False,False
2,luciano-pizzeria-ristorante-northwest-raleigh,luciano pizzeria ristorante,False,5.0,2,Crabtree Valley Mall,27612,35.839681,-78.678791,$,...,True,False,False,False,False,False,False,False,False,False
3,olive-garden-italian-restaurant-wilson,olive garden italian restaurant,False,3.0,16,3401 Raleigh Rd Pkwy W,27893,35.743170,-77.964160,$$,...,True,False,False,False,False,True,False,False,False,False
4,subway-wilson-4,subway,False,3.5,3,4934 Raleigh Rd,27896,35.762936,-77.999428,$,...,False,False,False,False,False,False,False,False,False,False


In [60]:
df_violations = pd.read_csv('/Users/alanmc/Desktop/data/violations.csv')
df_violations.head()

,X.objectid,hsisid,inspectdate,category,statecode,critical,questionno,violationcode,severity,shortdesc,inspectedby,comments,pointvalue,observationtype,violationtype,count,cdcriskfactor,cdcdataitem
0,2149,4092015279,2014-09-22T00:00:00.000Z,Chemical,".2653,.2657",Yes,26,7-102.11,Priority Foundation,"Toxic substances properly identified, stored, ...",Lucy Schrum,7-102.11; Priority Foundation - Found unlabele...,0,Out,R,NaN,NaN,NaN
1,2150,4092014572,2014-09-29T00:00:00.000Z,Chemical,".2653,.2657",Yes,26,7-102.11,Priority Foundation,"Toxic substances properly identified, stored, ...",Daryl Beasley,7-102.11; Priority Foundation; One sanitizer b...,0,Out,CDI,NaN,NaN,NaN
2,2151,4092015906,2014-10-01T00:00:00.000Z,Chemical,".2653,.2657",Yes,26,7-102.11,Priority Foundation,"Toxic substances properly identified, stored, ...",Lucy Schrum,7-102.11; Priority Foundation - Found an unlab...,1,Out,CDI,NaN,NaN,NaN
3,2152,4092013840,2014-10-08T00:00:00.000Z,Chemical,".2653,.2657",Yes,26,7-102.11,Priority Foundation,"Toxic substances properly identified, stored, ...",Lucy Schrum,7-102.11; Priority Foundation - Found unlabele...,0,Out,CDI,NaN,NaN,NaN
4,2153,4092021788,2014-10-09T00:00:00.000Z,Chemical,".2653,.2657",Yes,26,7-102.11,Priority Foundation,"Toxic substances properly identified, stored, ...",Lucy Schrum,7-102.11; Priority Foundation - Found one unla...,0,Out,CDI,NaN,NaN,NaN


In [57]:
df_restaurants = pd.read_csv('/Users/alanmc/Desktop/data/restaurants.csv')
df_restaurants.head()

,X.objectid,hsisid,name,address1,address2,city,state,postalcode,phonenumber,restaurantopendate,facilitytype,x,y,geocodestatus
0,1001,4092017230,SPRING CAFE 2,2900-104 SPRING FOREST RD,NaN,RALEIGH,NC,27616-1895,(919) 977-3679,2016-05-26T00:00:00.000Z,Restaurant,-78.591634,35.855487,M
1,1002,4092040338,CAROLINA CLASSIC HOT DOGS #2 (WCID #549),309 HOLLOMAN ST,NaN,APEX,NC,27502,NaN,2016-07-01T00:00:00.000Z,Pushcarts,-78.855348,35.730219,M
2,1003,4092014444,Taco Bell #22798,2207 S MAIN ST,NaN,WAKE FOREST,NC,27587,(919) 554-4924,2005-12-05T00:00:00.000Z,Restaurant,-78.536145,35.946695,M
3,1004,4092015333,THE REMEDY DINER,137 E HARGETT ST,NaN,RALEIGH,NC,27601,(919) 835-3553,2009-02-04T00:00:00.000Z,Restaurant,-78.636895,35.777999,M
4,1005,4092160069,ZEBULON HOUSE (KITCHEN),551 PONY RD,NaN,ZEBULON,NC,27597,NaN,2009-02-18T00:00:00.000Z,Institutional Food Service,-78.332138,35.816779,M


# Data Cleaning

Describe your data cleaning steps here.

In [58]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

In [61]:
df_rest=pd.merge( df_insp,df_restaurants, on="hsisid")
df_rest.head()

,hsisid,date,name_x,address1_x,address2_x,city_x,state_x,postalcode_x,phonenumber_x,restaurantopendate_x,...,address2_y,city_y,state_y,postalcode_y,phonenumber_y,restaurantopendate_y,facilitytype_y,x_y,y_y,geocodestatus_y
0,4092013748,2012-09-21T00:00:00Z,Cafe 3000 At Wake Med,3000 New Bern Ave,NaN,raleigh,NC,27610,(919) 350-8047,2002-12-21T00:00:00Z,...,NaN,RALEIGH,NC,27610,(919) 350-8047,2002-12-21T00:00:00.000Z,Restaurant,-78.587963,35.783914,M
1,4092013748,2013-02-14T00:00:00Z,Cafe 3000 At Wake Med,3000 New Bern Ave,NaN,raleigh,NC,27610,(919) 350-8047,2002-12-21T00:00:00Z,...,NaN,RALEIGH,NC,27610,(919) 350-8047,2002-12-21T00:00:00.000Z,Restaurant,-78.587963,35.783914,M
2,4092013748,2013-08-08T00:00:00Z,Cafe 3000 At Wake Med,3000 New Bern Ave,NaN,raleigh,NC,27610,(919) 350-8047,2002-12-21T00:00:00Z,...,NaN,RALEIGH,NC,27610,(919) 350-8047,2002-12-21T00:00:00.000Z,Restaurant,-78.587963,35.783914,M
3,4092013748,2014-04-03T00:00:00Z,Cafe 3000 At Wake Med,3000 New Bern Ave,NaN,raleigh,NC,27610,(919) 350-8047,2002-12-21T00:00:00Z,...,NaN,RALEIGH,NC,27610,(919) 350-8047,2002-12-21T00:00:00.000Z,Restaurant,-78.587963,35.783914,M
4,4092013748,2014-10-03T00:00:00Z,Cafe 3000 At Wake Med,3000 New Bern Ave,NaN,raleigh,NC,27610,(919) 350-8047,2002-12-21T00:00:00Z,...,NaN,RALEIGH,NC,27610,(919) 350-8047,2002-12-21T00:00:00.000Z,Restaurant,-78.587963,35.783914,M


In [ ]:
df_rest=pd.merge( df_violations,df_restaurants, on="X.objectid")
print(df_rest['hsisid_x'],df_rest['hsisid_y'])

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

This information that I will be looking for is public hence I will have permission to use it since it is our right to know the grade of each restaurant’s food inspection. There are no privacy concerns regarding the datasets since it is public information. The data could have potential biases by the inspectors, as mention in the reference article (3), inspectors can be biased on how many violations they mark in a restaurant and this can affect the dataset. This could be an ethical problem since an inspector can affect a restaurant’s reputation, therefore we expect that each inspector’s review is not biased. For the issue of finding inspector biases what can be done is to assign two inspectors to do the same reviews of restaurants and look at both data sets. This will allow us to see if there are any biases in the process of an inspector. In case we see a discrepancy between two inspectors we can ask a third inspector to review the particular restaurant. This will only be able if the Department of Environmental Health of San Diego works closely with us. But this will help prevent collection bias in our process.

# Conclusion & Discussion

*Fill in your discussion information here*